In [1]:
!pip install faker

In [2]:
import pandas as pd
import numpy as np
from faker import Faker

# Faker 인스턴스 생성
fake = Faker()

# fake 직원 데이터 생성
num_employees = 100
data = {
    'EmployeeID': np.arange(1, num_employees + 1),
    'FirstName': [fake.first_name() for _ in range(num_employees)],
    'LastName': [fake.last_name() for _ in range(num_employees)],
    'Salary': [round(fake.random_number(digits=5), 2) for _ in range(num_employees)],
    'YearsAtCompany': [fake.random_int(min=1, max=20) for _ in range(num_employees)]
}

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
csv_path = 'employees.csv'
df.to_csv(csv_path, index=False)

print(f'{csv_path} 파일이 생성되었습니다.')


employees.csv 파일이 생성되었습니다.


In [4]:
df.tail()

,EmployeeID,FirstName,LastName,Salary,YearsAtCompany
95,96,Kevin,Villegas,68963,1
96,97,David,Nelson,64883,13
97,98,Jason,Ashley,4353,13
98,99,Shannon,Chang,10506,12
99,100,Tyler,Warner,63257,20


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import pandas as pd
import numpy as np
import sqlite3
from faker import Faker
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI
from langchain.agents.agent_types import AgentType

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os

In [11]:
# SQLite 데이터베이스로 로드
db_path = 'employees.db'

def load_csv_to_db(csv_path, db_path):
    df = pd.read_csv(csv_path)
    conn = sqlite3.connect(db_path)
    df.to_sql('data', conn, if_exists='replace', index=False)
    conn.close()

load_csv_to_db(csv_path, db_path)

# OpenAI LLM 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# SQLite 데이터베이스 연결 설정
db = sqlite3.connect(db_path)

In [10]:
# SQL 에이전트 생성
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True
)

ValidationError: 1 validation error for SQLDatabaseToolkit
db
  instance of SQLDatabase expected (type=type_error.arbitrary_type; expected_arbitrary_type=SQLDatabase)

In [ ]:




# 사용자 정의 회귀 분석 함수
def perform_regression(inputs):
    # SQLite 데이터베이스에서 데이터 로드
    df = pd.read_sql_query(inputs['query'], conn)
    
    # 데이터 전처리
    features = df[['Salary', 'YearsAtCompany']].values  # 모델 입력 피처
    target = df['YearsAtCompany'].values  # 모델 출력 피처
    
    # 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # 회귀 모델 생성 및 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 모델 평가 및 예측
    score = model.score(X_test, y_test)
    predictions = model.predict(X_test)
    
    return {
        'score': score,
        'predictions': predictions.tolist()
    }

# LangChain 에이전트 실행기 설정
class CustomAgent:
    def __init__(self, llm, sql_agent, regression_func):
        self.llm = llm
        self.sql_agent = sql_agent
        self.regression_func = regression_func

    def run(self, inputs):
        # SQL 쿼리 생성
        sql_query_response = self.sql_agent.run({"input": inputs["input"]})
        sql_query = sql_query_response["sql_query"]

        # 회귀 분석 수행
        regression_results = self.regression_func({"query": sql_query})

        return {
            "sql_query": sql_query,
            "regression_results": regression_results
        }

# 사용자 정의 에이전트 인스턴스 생성
custom_agent = CustomAgent(llm, sql_agent, perform_regression)

# 자연어 입력을 통해 에이전트 실행
response = custom_agent.run({"input": "급여와 근무 기간을 바탕으로 회귀 분석을 수행해 주세요."})

# 결과 출력
print(response)
